# 약물-단백질 상호작용

- 약물과 단백질의 상호작용 정도를 점수화한 데이터를 이용하여 상호작용을 예측하는 머신러닝 모델을 구현
- 약물과 단백질 데이터를 각각 입력으로 사용한다
    1. 약물의 특성과 단백질(타겟)의 특성을 각각 추출하는 모델 구성
    2. 추출된 특성을 **옆으로 붙인(concatenate)** 값을 만드고 이를 DTI 예측 모델 입력으로 사용한다
    3. 상호작용은 이진분류(0/1) 예측(확률) 또는 상호작용 점수 회귀 예측을 하게 된다.
- 약물 재창출 예제

- [DeepPurpose](https://deeppurpose.readthedocs.io/en/latest/notes/introduction.html) 모듈 사용
 - 약물-단백질 상호작용 예측 모델 구현 기능을 제공
 - 가상선별과 약물재창출을 위한 딥러닝 도구

# import

In [2]:
import numpy as np
from DeepPurpose.utils import *

## 데이터 불러오기

In [3]:
# 파일 이름 목록 생성
file_names = [f"./data/DAVIS_X_target_part{i}.npy" for i in range(15)]

# 각 파일을 읽어와 리스트에 저장
arrays = [np.load(file) for file in file_names]

# 리스트에 저장된 모든 어레이를 하나로 합침
X_target = np.concatenate(arrays)

X_drug = np.load('./data/DAVIS_X_drug.npy')
y = np.load('./data/DAVIS_y.npy')

In [4]:
# 화합물 데이터 확인
print("drug : ", X_drug[:3], end="\n\n")

# 타겟 단백질 데이터 확인
print("target : ",X_target[0], end="\n\n")

# 결합 여부 확인 (0이면 결합 안함, 1이면 결합)
print("Interaction : ",y[:3], end="\n\n")

drug :  ['CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N'
 'CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N'
 'CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N']

target :  MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFG

## MLP 모델 입력에 맞도록 전처리 (DeepPurpose의 data_process함수 사용)

In [19]:
# MLP 모델을 위한 encoding 방식 지정 (입력 특성으로 1차원 벡터를 사용)
drug_encoding = 'Morgan'
target_encoding = 'AAC'
train, val, test = data_process(X_drug, X_target, y,
                                drug_encoding, target_encoding,
                                split_method='random', frac=[0.8,0.1,0.1])
train[:3]

Drug Target Interaction Prediction Mode...
in total: 30056 drug-target pairs
encoding drug...
unique drugs: 68
encoding protein...
unique target sequence: 379
-- Encoding AAC takes time. Time Reference: 24s for ~100 sequences in a CPU.				 Calculate your time by the unique target sequence #, instead of the entire dataset.
splitting dataset...
Done.


,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,0,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8.221, 3.33, 3.434, 4.683, 1.353, 4.475, 11.6..."
1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,0,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8.483, 5.827, 3.599, 3.685, 1.028, 7.798, 3.0..."
2,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,0,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8.483, 5.827, 3.599, 3.685, 1.028, 7.798, 3.0..."


## CNN 모델 입력에 맞도록 전처리 (DeepPurpose의 data_process함수 사용)

In [20]:
drug_encoding = 'CNN'
target_encoding = 'CNN'
train, val, test = data_process(X_drug, X_target, y,
                                drug_encoding, target_encoding,
                                split_method='random',frac=[0.8,0.1,0.1])
train[:3]

Drug Target Interaction Prediction Mode...
in total: 30056 drug-target pairs
encoding drug...
unique drugs: 68
encoding protein...
unique target sequence: 379
splitting dataset...
Done.


,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,0,"[C, C, 1, =, C, 2, C, =, C, (, C, =, C, C, 2, ...","[M, K, K, F, F, D, S, R, R, E, Q, G, G, S, G, ..."
1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,0,"[C, C, 1, =, C, 2, C, =, C, (, C, =, C, C, 2, ...","[P, F, W, K, I, L, N, P, L, L, E, R, G, T, Y, ..."
2,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,0,"[C, C, 1, =, C, 2, C, =, C, (, C, =, C, C, 2, ...","[P, F, W, K, I, L, N, P, L, L, E, R, G, T, Y, ..."
